In [1]:

%pprint
import sys
if ('../py' not in sys.path): sys.path.insert(1, '../py')

Pretty printing has been turned OFF


In [2]:

from FRVRS import (fu, nu, np, osp, display)

In [3]:

# Get all CSVs into one data frame
if nu.pickle_exists('frvrs_logs_df'):
    frvrs_logs_df = nu.load_object('frvrs_logs_df')
    print(frvrs_logs_df.shape)
    # df = frvrs_logs_df.sample(4).dropna(axis='columns', how='all')
    # display(df.T)

(829277, 113)



The _TOOL_APPLIED_ log entry contains no **injury_id**, and the attachment point is not even unambiguously mapped to a body region, so, given that some patients have multiple injuries, we are basically hoping that the required procedure existing among the treated injuries means that the correct tool was applied.


## Check for Doubles

In [4]:

df = nu.check_4_doubles(frvrs_logs_df.tool_applied_attachment_point.unique())
column_list = ['first_item', 'second_item', 'max_similarity']
mask_series = (df.max_similarity > 0.95)
display(df[mask_series][column_list].sort_values('max_similarity', ascending=False))

,first_item,second_item,max_similarity
5,skinCollider_BodyLOD (UnityEngine.GameObject),skinCollider_Body_LOD (UnityEngine.GameObject),0.989011
1,RChest_Breathe1 (UnityEngine.GameObject),LChest_Breathe1 (UnityEngine.GameObject),0.975000
4,RightUpLeg (UnityEngine.GameObject),RightLeg (UnityEngine.GameObject),0.970588
2,LeftUpLeg (UnityEngine.GameObject),LeftLeg (UnityEngine.GameObject),0.969697
6,skinCollider_Body_LOD (UnityEngine.GameObject),skinCollider_BodyGar_LOD (UnityEngine.GameObject),0.968421


In [5]:

df = nu.check_4_doubles(frvrs_logs_df.injury_record_body_region.unique())
column_list = ['first_item', 'second_item', 'max_similarity']
mask_series = (df.max_similarity > 0.5)
display(df[mask_series][column_list].sort_values('max_similarity', ascending=False))

,first_item,second_item,max_similarity
3,rightArm,rightLeg,0.625000
2,leftLeg,leftArm,0.571429


In [6]:

df = nu.check_4_doubles(frvrs_logs_df.injury_treated_body_region.unique())
column_list = ['first_item', 'second_item', 'max_similarity']
mask_series = (df.max_similarity > 0.5)
display(df[mask_series][column_list].sort_values('max_similarity', ascending=False))

,first_item,second_item,max_similarity
3,rightArm,rightLeg,0.625000
2,leftLeg,leftArm,0.571429


In [7]:

df = nu.check_4_doubles(frvrs_logs_df.injury_id.unique())
column_list = ['first_item', 'second_item', 'max_similarity']
mask_series = (df.max_similarity > 0.9)
display(df[column_list][mask_series].sort_values('max_similarity', ascending=False))

,first_item,second_item,max_similarity
4,R Forearm Laceration,L Forearm Laceration,0.950000
2,L Shoulder Puncture,R Shoulder Puncture,0.947368
3,L Thigh Laceration,R Thigh Laceration,0.944444
18,L Wrist Amputation,R Wrist Amputation,0.944444
11,R Calf Laceration,L Calf Laceration,0.941176
13,R Shin Amputation,L Shin Amputation,0.941176
1,R Chest Collapse,L Chest Collapse,0.937500
20,R Bicep Puncture,L Bicep Puncture,0.937500
5,R Side Puncture,L Side Puncture,0.933333
7,R Calf Shrapnel,L Calf Shrapnel,0.933333



## Attachment Point

In [8]:

left_mask_series = ~frvrs_logs_df.tool_applied_attachment_point.isnull()
right_mask_series = ~frvrs_logs_df.injury_record_body_region.isnull()
df = nu.check_for_typos(
    np.sort(
        frvrs_logs_df[left_mask_series].tool_applied_attachment_point.unique()
    ), frvrs_logs_df[right_mask_series].injury_record_body_region.unique(),
    rename_dict={'left_item': 'attachment_point', 'right_item': 'body_region'}
)
display(df)

,attachment_point,body_region,max_similarity
0,LChest_Breathe1 (UnityEngine.GameObject),chest,0.177778
1,LeftArm (UnityEngine.GameObject),leftArm,0.307692
2,LeftForeArm (UnityEngine.GameObject),leftArm,0.279070
3,LeftLeg (UnityEngine.GameObject),leftLeg,0.307692
4,LeftUpLeg (UnityEngine.GameObject),leftLeg,0.292683
5,RChest_Breathe1 (UnityEngine.GameObject),chest,0.177778
6,RightArm (UnityEngine.GameObject),rightArm,0.341463
7,RightForeArm (UnityEngine.GameObject),rightArm,0.311111
8,RightLeg (UnityEngine.GameObject),rightLeg,0.341463
9,RightUpLeg (UnityEngine.GameObject),rightLeg,0.325581


In [9]:

left_mask_series = ~frvrs_logs_df.tool_applied_attachment_point.isnull()
right_mask_series = ~frvrs_logs_df.injury_id.isnull()
df = nu.check_for_typos(
    np.sort(frvrs_logs_df[left_mask_series].tool_applied_attachment_point.unique()), frvrs_logs_df[right_mask_series].injury_id.unique(),
    rename_dict={'left_item': 'attachment_point', 'right_item': 'injury_id'}
)
display(df)

,attachment_point,injury_id,max_similarity
0,LChest_Breathe1 (UnityEngine.GameObject),L Calf Shrapnel,0.254545
1,LeftArm (UnityEngine.GameObject),L Stomach Puncture,0.280000
2,LeftForeArm (UnityEngine.GameObject),L Forearm Laceration,0.321429
3,LeftLeg (UnityEngine.GameObject),L Side Puncture,0.255319
4,LeftUpLeg (UnityEngine.GameObject),L Bicep Puncture,0.280000
5,RChest_Breathe1 (UnityEngine.GameObject),R Calf Shrapnel,0.254545
6,RightArm (UnityEngine.GameObject),L Thigh Puncture,0.285714
7,RightForeArm (UnityEngine.GameObject),R Forearm Laceration,0.315789
8,RightLeg (UnityEngine.GameObject),R Side Puncture,0.291667
9,RightUpLeg (UnityEngine.GameObject),R Side Puncture,0.280000



## Body Region

In [10]:

left_mask_series = ~frvrs_logs_df.injury_record_body_region.isnull()
right_mask_series = ~frvrs_logs_df.tool_applied_attachment_point.isnull()
df = nu.check_for_typos(
    np.sort(
        frvrs_logs_df[left_mask_series].injury_record_body_region.unique()
    ), frvrs_logs_df[right_mask_series].tool_applied_attachment_point.unique(),
    rename_dict={'left_item': 'body_region', 'right_item': 'attachment_point'}
)
display(df)

,body_region,attachment_point,max_similarity
0,abdomen,Spine1 (UnityEngine.GameObject),0.157895
1,chest,RChest_Breathe1 (UnityEngine.GameObject),0.177778
2,head,RightArm (UnityEngine.GameObject),0.162162
3,leftArm,LeftArm (UnityEngine.GameObject),0.307692
4,leftLeg,LeftLeg (UnityEngine.GameObject),0.307692
5,neck,Spine1 (UnityEngine.GameObject),0.171429
6,rightArm,RightArm (UnityEngine.GameObject),0.341463
7,rightLeg,RightLeg (UnityEngine.GameObject),0.341463


In [11]:

left_mask_series = ~frvrs_logs_df.injury_record_body_region.isnull()
right_mask_series = ~frvrs_logs_df.injury_id.isnull()
df = nu.check_for_typos(
    np.sort(frvrs_logs_df[left_mask_series].injury_record_body_region.unique()), frvrs_logs_df[right_mask_series].injury_id.unique(),
    rename_dict={'left_item': 'body_region', 'right_item': 'injury_id'}
)
display(df)

,body_region,injury_id,max_similarity
0,abdomen,Face Shrapnel,0.300000
1,chest,R Chest Collapse,0.380952
2,head,Forehead Scrape,0.421053
3,leftArm,L Shoulder Puncture,0.307692
4,leftLeg,Ear Bleed,0.375000
5,neck,L Neck Puncture,0.315789
6,rightArm,L Thigh Puncture,0.416667
7,rightLeg,L Thigh Puncture,0.416667



## Injury ID

In [12]:

left_mask_series = ~frvrs_logs_df.injury_id.isnull()
right_mask_series = ~frvrs_logs_df.tool_applied_attachment_point.isnull()
df = nu.check_for_typos(
    np.sort(frvrs_logs_df[left_mask_series].injury_id.unique()), frvrs_logs_df[right_mask_series].tool_applied_attachment_point.unique(),
    rename_dict={'left_item': 'injury_id', 'right_item': 'attachment_point'}
)
display(df)

,injury_id,attachment_point,max_similarity
0,Asthmatic,RChest_Breathe1 (UnityEngine.GameObject),0.244898
1,Ear Bleed,Spine1 (UnityEngine.GameObject),0.200000
2,Face Shrapnel,LeftForeArm (UnityEngine.GameObject),0.204082
3,Forehead Scrape,LeftForeArm (UnityEngine.GameObject),0.274510
4,L Bicep Puncture,LeftArm (UnityEngine.GameObject),0.250000
5,L Calf Laceration,LChest_Breathe1 (UnityEngine.GameObject),0.280702
6,L Calf Shrapnel,Spine1 (UnityEngine.GameObject),0.173913
7,L Chest Collapse,LChest_Breathe1 (UnityEngine.GameObject),0.321429
8,L Forearm Laceration,LeftForeArm (UnityEngine.GameObject),0.392857
9,L Neck Puncture,LeftArm (UnityEngine.GameObject),0.212766


In [13]:

left_mask_series = ~frvrs_logs_df.injury_id.isnull()
right_mask_series = ~frvrs_logs_df.injury_record_body_region.isnull()
df = nu.check_for_typos(
    np.sort(frvrs_logs_df[left_mask_series].injury_id.unique()), frvrs_logs_df[right_mask_series].injury_record_body_region.unique(),
    rename_dict={'left_item': 'injury_id', 'right_item': 'body_region'}
)
display(df)

,injury_id,body_region,max_similarity
0,Asthmatic,head,0.307692
1,Ear Bleed,leftLeg,0.375000
2,Face Shrapnel,abdomen,0.300000
3,Forehead Scrape,head,0.421053
4,L Bicep Puncture,chest,0.285714
5,L Calf Laceration,chest,0.272727
6,L Calf Shrapnel,leftArm,0.272727
7,L Chest Collapse,chest,0.380952
8,L Forearm Laceration,abdomen,0.222222
9,L Neck Puncture,neck,0.315789


In [14]:

# Create a mask series that only includes rows where the action_type is equal to TOOL_APPLIED
mask_series = (frvrs_logs_df.action_type == 'TOOL_APPLIED')

# Group the data frame by the patient group by columns using the groupby() function
gb = frvrs_logs_df[mask_series].sort_values(['action_tick']).groupby(fu.patient_groupby_columns)

# Sort the groups by the number of rows in each group using the sorted() function
tool_applieds_df_with_most_rows_key = sorted([(k, v) for k, v in gb.groups.items()], key=lambda x: len(x[1]), reverse=True)[0][0]
tool_applieds_df_with_most_rows_key

('16373b36-886a-4d0d-9015-474d29d139cc', 0, 'Mike_5 Root')

In [16]:

# for (i, v), cn in zip(enumerate(tool_applieds_df_with_most_rows_key), fu.patient_groupby_columns):
#     print(f"mask_series &= (frvrs_logs_df.{cn} == '{v}')")
patient_mask_series = (frvrs_logs_df.session_uuid == tool_applieds_df_with_most_rows_key[0])
patient_mask_series &= (frvrs_logs_df.scene_id == tool_applieds_df_with_most_rows_key[1])
patient_mask_series &= (frvrs_logs_df.patient_id == tool_applieds_df_with_most_rows_key[2])

mask_series = patient_mask_series & (frvrs_logs_df.action_type == 'TOOL_APPLIED')
print(frvrs_logs_df[mask_series].shape)
print(nu.conjunctify_nouns(frvrs_logs_df[mask_series].tool_applied_type.unique()))
print(nu.conjunctify_nouns(frvrs_logs_df[mask_series].tool_applied_attachment_point.unique()))
print(nu.conjunctify_nouns(frvrs_logs_df[mask_series].tool_applied_sender.unique()))
display(frvrs_logs_df[mask_series].sample(6).dropna(axis='columns', how='all').T)

mask_series = patient_mask_series & (frvrs_logs_df.action_type == 'INJURY_RECORD')
print(frvrs_logs_df[mask_series].shape)
print(nu.conjunctify_nouns(frvrs_logs_df[mask_series].injury_record_id.unique()))
print(nu.conjunctify_nouns(frvrs_logs_df[mask_series].injury_record_required_procedure.unique()))
print(nu.conjunctify_nouns(frvrs_logs_df[mask_series].injury_record_body_region.unique()))
frvrs_logs_df[mask_series].dropna(axis='columns', how='all').T

(10, 113)
Gauze_Dressing, Tourniquet, and Gauze_Pack
skinCollider_BodyCollideLOD (UnityEngine.GameObject) and RightUpLeg (UnityEngine.GameObject)
AppliedDressingGauze, AppliedTourniquet, and AppliedPackingGauze


,644016,644054,643877,644104,644136,644159
action_type,TOOL_APPLIED,TOOL_APPLIED,TOOL_APPLIED,TOOL_APPLIED,TOOL_APPLIED,TOOL_APPLIED
action_tick,344666,356578,329767,384818,395289,397712
event_time,2022-03-15 13:20:00,2022-03-15 13:21:00,2022-03-15 13:20:00,2022-03-15 13:21:00,2022-03-15 13:21:00,2022-03-15 13:21:00
session_uuid,16373b36-886a-4d0d-9015-474d29d139cc,16373b36-886a-4d0d-9015-474d29d139cc,16373b36-886a-4d0d-9015-474d29d139cc,16373b36-886a-4d0d-9015-474d29d139cc,16373b36-886a-4d0d-9015-474d29d139cc,16373b36-886a-4d0d-9015-474d29d139cc
file_name,Double runs removed/22.03.15.1315r.csv,Double runs removed/22.03.15.1315r.csv,Double runs removed/22.03.15.1315r.csv,Double runs removed/22.03.15.1315r.csv,Double runs removed/22.03.15.1315r.csv,Double runs removed/22.03.15.1315r.csv
logger_version,1.0,1.0,1.0,1.0,1.0,1.0
tool_applied_type,Gauze_Dressing,Gauze_Pack,Gauze_Dressing,Gauze_Pack,Gauze_Pack,Gauze_Dressing
tool_applied_attachment_point,skinCollider_BodyCollideLOD (UnityEngine.GameO...,skinCollider_BodyCollideLOD (UnityEngine.GameO...,skinCollider_BodyCollideLOD (UnityEngine.GameO...,skinCollider_BodyCollideLOD (UnityEngine.GameO...,skinCollider_BodyCollideLOD (UnityEngine.GameO...,skinCollider_BodyCollideLOD (UnityEngine.GameO...
tool_applied_sender,AppliedDressingGauze,AppliedPackingGauze,AppliedDressingGauze,AppliedPackingGauze,AppliedPackingGauze,AppliedDressingGauze
tool_applied_patient_id,Mike_5 Root,Mike_5 Root,Mike_5 Root,Mike_5 Root,Mike_5 Root,Mike_5 Root


(2, 113)
R Side Puncture and R Thigh Laceration
woundpack and gauzePressure
abdomen and rightLeg


,643457,643458
action_type,INJURY_RECORD,INJURY_RECORD
action_tick,178131,178131
event_time,2022-03-15 13:18:00,2022-03-15 13:18:00
session_uuid,16373b36-886a-4d0d-9015-474d29d139cc,16373b36-886a-4d0d-9015-474d29d139cc
file_name,Double runs removed/22.03.15.1315r.csv,Double runs removed/22.03.15.1315r.csv
logger_version,1.0,1.0
injury_record_id,R Side Puncture,R Thigh Laceration
injury_record_patient_id,Mike_5 Root,Mike_5 Root
injury_record_required_procedure,woundpack,gauzePressure
injury_record_severity,medium,medium
